# Домашнее задание № 4. Языковые модели

## Задание 1 (8 баллов).

В семинаре для генерации мы использовали предположение маркова и считали, что слово зависит только от 1 предыдущего слова. Но ничто нам не мешает попробовать увеличить размер окна и учитывать два или даже три прошлых слова. Для них мы еще сможем собрать достаточно статистик и, логично предположить, что качество сгенерированного текста должно вырасти.

Попробуйте сделать языковую модель, которая будет учитывать два предыдущих слова при генерации текста.
Сгенерируйте несколько текстов (3-5) и расчитайте перплексию получившейся модели. 
Можно использовать данные из семинара или любые другие (сопоставимые или большие по объему). Перплексию рассчитывайте на 10-50 отложенных предложениях (они не должны использоваться при сборе статистик).


Подсказки:  
    - нужно будет добавить еще один тэг <start>  
    - еще одна матрица не нужна, можно по строкам хронить биграмы, а по колонкам униграммы  
    - тексты должны быть очень похожи на нормальные (если у вас получается рандомная каша, вы что-то делаете не так). 

In [2]:
from collections import Counter
from nltk.tokenize import sent_tokenize
from razdel import tokenize as razdel_tokenize
from string import punctuation
import numpy as np
from scipy.sparse import csr_matrix, csc_matrix, lil_matrix

In [3]:
news = open('lenta.txt', encoding='utf8').read()
def normalize(text):
    normalized_text = [word.text.strip(punctuation) for word \
                                                            in razdel_tokenize(text)]
    normalized_text = [word.lower() for word in normalized_text if word and len(word) < 20 ]
    return normalized_text
norm_news = normalize(news)

def ngrammer(tokens, n=2):
    ngrams = []
    for i in range(0,len(tokens)-n+1):
        ngrams.append(' '.join(tokens[i:i+n]))
    return ngrams

In [4]:
sentences_news = [['<start>'] + ['<start>'] + normalize(text) +['<end>'] for text in sent_tokenize(news)]

unigrams_news = Counter()
bigrams_news = Counter()
trigrams_news = Counter()
i = 0
for sentence in sentences_news:
    unigrams_news.update(sentence)
    bigrams_news.update(ngrammer(sentence))
    trigrams_news.update(ngrammer(sentence, 3))
    if i >5000:
        break
    i+=1

len(unigrams_news)

24039

In [5]:
len(bigrams_news)

75188

In [6]:
bigrams_news.most_common(10)


[('<start> <start>', 5002),
 ('<start> в', 450),
 ('<start> по', 393),
 ('<start> как', 274),
 ('риа новости', 171),
 ('<start> на', 126),
 ('по словам', 126),
 ('в москве', 110),
 ('что в', 108),
 ('об этом', 107)]

In [7]:
trigrams_news.most_common(10)

[('<start> <start> в', 450),
 ('<start> <start> по', 393),
 ('<start> <start> как', 274),
 ('<start> <start> на', 126),
 ('<start> <start> об', 99),
 ('<start> об этом', 95),
 ('<start> по словам', 94),
 ('<start> <start> однако', 94),
 ('<start> как сообщает', 88),
 ('<start> <start> он', 77)]

In [8]:
matrix_news = np.zeros((len(bigrams_news), 
                   len(unigrams_news)))
id2word_news_unigrams = list(unigrams_news)
word2id_news_unigrams = {word:i for i, word in enumerate(id2word_news_unigrams)}
id2word_news_bigrams = list(bigrams_news)
word2id_news_bigrams = {word:i for i, word in enumerate(id2word_news_bigrams)}
id2word_news_trigrams = list(trigrams_news)
word2id_news_trigrams = {word:i for i, word in enumerate(id2word_news_trigrams)}


In [9]:
for ngram in trigrams_news:
    word1, word2, word3 = ngram.split()
    bigram = word1 + ' ' + word2
    matrix_news[word2id_news_bigrams[bigram]][word2id_news_unigrams[word3]]= (trigrams_news[ngram]/bigrams_news[bigram])


In [10]:
def generate(matrix, id2word_unigram, word2id_unigram, id2word_bigram, word2id_bigram, n=100, start='<start> <start>'):
    text = []
    current_idx = word2id_bigram[start]
    
    for i in range(n):
        bigram = id2word_bigram[current_idx]
        first_word, second_word = bigram.split()
        chosen = np.random.choice(matrix.shape[1], p=matrix[current_idx])
        chosen_word = id2word_unigram[chosen]
        text.append(chosen_word)
        if chosen_word == '<end>':
            current_idx = word2id_bigram['<start> <start>']
        else:
            current_idx = word2id_bigram[second_word + ' '+ chosen_word]
    
    return ' '.join(text)

In [14]:
generated_text = generate(matrix_news, id2word_news_unigrams, word2id_news_unigrams, id2word_news_bigrams, word2id_news_bigrams, n=100).replace('<end>', '\n')

In [15]:
print(generated_text)

к террористической группе осуществившей взрывы жилых домов в столице карачаево-черкесии городе черкесске 
 в этой связи юрий лужков и впредь 
 властям удалось взять ситуацию под контролем обсе 
 оно ощущалось как на сопровождаемый так и не локализован 
 на прошлой неделе была найдена запись на которой агенты фбр признали что использовали в уэйко и найти того кто не побоится нас защитить 
 специалисты гостелекома отмечают что новый блок первое объединение инициатива создания которого исходит непосредственно из регионов 
 тогда же прибыли венки от президента и возглавил орт 
 последние события в дагестане террористические акты в двух местных кафе 



In [25]:
def perplexity(probas):
    p = np.exp(np.sum(probas))
    N = len(probas)
    
    return p**(-1/N) 

In [26]:
norm_generated_text= normalize(generated_text)

In [27]:
vocab_generated_text = Counter(norm_generated_text)

In [28]:
probas_generated_text = Counter({word:c/len(generated_text) for word, c in vocab_generated_text.items()})

In [31]:
prob = {'generated_text':[]}

for word in normalize(generated_text):
    
    if word in probas_generated_text:
        prob['generated_text'].append(np.log(probas_generated_text[word]))
    else:
        prob['generated_text'].append(np.log(1/len(norm_generated_text)))

In [32]:
perplexity(prob['generated_text'])

528.1620154711247

## Задание № 2* (2 балла). 

Прочитайте главу про языковое моделирование в книге Журафски и Мартина - https://web.stanford.edu/~jurafsky/slp3/3.pdf

Развернуто (в пределах 1000 знаков) ответьте на вопросы (по-русски):

1. Что можно делать с проблемой несловарных слов? В семинаре мы просто использовали какое-то маленькое значение вероятности, а какие есть другие способы?

Есть два способы, оба с тэгом <UNK> (unknown). 
Первый способ принимается когда изначально есть список правильных слов. В процессе нормализации, слова, которых нет в списке заменяются на тэг <UNK>, и потом тэг вычисляется как другие слова. 
Второй способ принимается когда нет списка правильных слов изначально. В этом методе, самые нечастые слова заменяются на тэг <UNK>, и как в первом методе, тэг вычисляется как другие слова. Можно или заменять такие слова, которые встречаются меньше n раз, или решить что словарь состоится от V слов, и не заменять самые частные V слова, а заменять остальные.  

2. Что такое сглаживание (smoothing)?

Сглаживание это процесс, который дает маленькая вероятность к случаям, которых не были в тренировочных данных. Это делается потому что, такие случаи могут встречаться в тестовых данных. 
Есть разные способы реализовать сглаживание. Например, в сглаживании Лапласа, каждому случаю добавляется один. Но это дает слишком много вероятности нулям, и поэтому можно добавить число, меньше одного, как 0.5 или 0.1. Ещё можно вычислять вероятность используя более маленькие ngram (например, если вычисляете вероятность триграмы, добавить вероятность биграм или униграм) Ещё можно вычитать какое-то абсолютное значение, как в Kneser-Nay. Ещё можно вычислять вероятность того, что слова находится в новом контексте, на основании того, в скольких разных контекстах это слово было. 